Setting runtime environment

In [ ]:
RUN_FROM = 'LOCAL'
# RUN_FROM = 'COLAB'

if RUN_FROM == 'LOCAL':
    from os.path import expanduser
    HOME = expanduser("~")
elif RUN_FROM == 'COLAB':
    !pip install hanziconv
    from google.colab import drive
    drive.mount('/content/drive')
    %cd /content/drive/My Drive/projects/lm
    HOME = '/content/drive/My Drive'

Import modules

In [ ]:
import matplotlib.pyplot as plt
import os, sys, time
import tensorflow as tf
from model import file_input_fn, lm_model_fn, lr_range_test, train_and_evaluate
from utils import data_utils_py3

Setting parameters

In [ ]:
char_vocab_file = HOME+"/Data/Vocab/vocab_zh"
word_vocab_file = HOME+"/Data/Vocab/word_vocab_zh"
char_embedding_files = HOME+"/Data/Vocab/zh_char_300_nlpcc.txt"
data_dir = HOME+"/Data/text_zh"
if RUN_FROM == 'LOCAL':
    train_dir = 'pretrain_zh_small'
elif RUN_FROM == 'COLAB':
    train_dir = HOME+'/Models/pretrain_zh_small'

char_vocab = data_utils_py3.Vocab(
    char_vocab_file,
    embedding_files=char_embedding_files)
char_vocab_size = char_vocab.size()

params = {
    'max_lr': 5e-5,
    'num_steps': 200000,
    'pct_start': 0.3,
    'dropout': 0.1,
    'wd': .0,
    'char_vocab_size': char_vocab_size,
    'char_vocab_dim': 300,
    'char_vocab_emb': char_vocab.embedding_init,
    'max_char_length': 8,
    'layer_size': 256,
    'num_layers': 3,
    'data_spec': [{'type': 'seq', 'is_target': True, 'select_from': None}]
}


batch_size = 32
num_pieces = 1
max_lengths = [256]
train_input_fn = lambda: file_input_fn(
    char_vocab, os.path.join(data_dir, 'train'),
    num_pieces, batch_size, max_lengths, True)
eval_input_fn = lambda: file_input_fn(
    char_vocab, os.path.join(data_dir, 'dev'),
    num_pieces, batch_size, max_lengths, False)

LR range test and plot the curve

In [ ]:
lr_range_test(lm_model_fn, train_input_fn, params)

Start train and eval loop

In [ ]:
train_and_evaluate(
    lm_model_fn, train_input_fn, eval_input_fn, train_dir, params, eval_every=10000)